# 🛠 Setup

In [ ]:
pip install -U "servicefoundry==0.6.6" "mlfoundry==0.6.1"

In [ ]:
import logging
[logging.root.removeHandler(h) for h in logging.root.handlers]
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(name)s] %(levelname)-8s %(message)s')

In [ ]:
!sfy login

In [ ]:
# Copy workspace FQN from https://app.truefoundry.com/workspaces
from getpass import getpass

WORKSPACE="tfy-cluster-euwe1:kotak-demo"
TFY_API_KEY = getpass("Please enter your API Key,( https://app.truefoundry.com/settings ):-")

# ⚡ Jobs - Quick Start
Here we will create a simple job that counts to 30 sleeping for 1 second every iteration.

In [ ]:
!mkdir -p /content/demo/simple/job
%cd /content/demo/simple/job

In [ ]:
%%writefile run.py

import argparse
import time


def main(limit: int):
    for i in range(limit):
        print(f"Running Job: {i + 1}")
        time.sleep(1)

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument("--limit", default=10, type=int)
    args = parser.parse_args()
    main(args.limit)

In [ ]:
import logging

from servicefoundry import Build, Job, PythonBuild, Resources

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s', force=True)

job = Job(
    name="counter-job",
    image=Build(build_spec=PythonBuild(command="python run.py")),
    resources=Resources(cpu_request=0.5, cpu_limit=1.0)
)
job.deploy(workspace_fqn=WORKSPACE)

# ☁ Service - Quick Start
Here we will take two python functions that use numpy and register them as routes in a deployed web service

In [ ]:
! mkdir -p /content/demo/simple/service
%cd /content/demo/simple/service

In [ ]:
%%writefile service.py
from fastapi import FastAPI

app = FastAPI()


@app.get("/")
async def root():
    return {"message": "Hello World"}

In [ ]:
%%writefile requirements.txt
fastapi==0.89.1
uvicorn==0.20.0

In [ ]:
import logging

from servicefoundry import Build, Service, PythonBuild, Resources

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s', force=True)

service = Service(
    name="fastapi-service",
    image=Build(build_spec=PythonBuild(command="uvicorn service:app --host 0.0.0.0 --port 8080")),
    resources=Resources(cpu_request=0.5, cpu_limit=1.0),
    ports=[{"port": 8080}]
)
service.deploy(workspace_fqn=WORKSPACE)

# ☁ Function as Service - Quick Start
Here we will take two python functions that use numpy and register them as routes in a deployed web service

In [ ]:
! mkdir -p /content/demo/simple/function-service
%cd /content/demo/simple/function-service

In [ ]:
%%writefile functions.py
from typing import List
import numpy as np

def multiply(a: float, b: float):
  return a * b 

def normal(loc: float, scale: float, size: List[int]):
    return np.random.normal(loc=loc, scale=scale, size=size).tolist()

def uniform(low: float, high: float, size: List[int]):
    return np.random.uniform(low=low, high=high, size=size).tolist()

In [ ]:
%%writefile requirements.txt
numpy==1.21.6

In [ ]:
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s', force=True)

from functions import normal, uniform, multiply
from servicefoundry.function_service import FunctionService

service = FunctionService(name="fn-service")
service.register_function(normal)
service.register_function(uniform)
service.register_function(multiply)
deployment = service.deploy(workspace_fqn=WORKSPACE)

In [ ]:
! mkdir -p /content/demo/regression/
%cd /content/demo/regression/

In [ ]:
%%writefile train.py
import argparse

import matplotlib.pyplot as plt
import mlfoundry
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import QuantileTransformer
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.metrics import PredictionErrorDisplay


def train(kernel: str):
    X, y = load_diabetes(as_frame=True, return_X_y=True)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    regressor = SVR(kernel=kernel)
    model = TransformedTargetRegressor(
        regressor=regressor,
        transformer=QuantileTransformer(n_quantiles=100, output_distribution="normal"),
    ).fit(X_train, y_train)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    PredictionErrorDisplay.from_predictions(
        y_test,
        y_pred,
        kind="actual_vs_predicted",
        scatter_kwargs={"alpha": 0.5},
    )

    run = mlfoundry.get_client().create_run(
        project_name="diabetes-regression", run_name="SVR-with-QT"
    )

    run.log_params(regressor.get_params())
    run.log_metrics({"score": model.score(X_test, y_test)})
    run.log_plots({"actual_vs_predicted": plt})

    model_version = run.log_model(
        name="diabetes-regression",
        model=model,
        framework="sklearn",
        description="SVC model trained on initial data",
        model_schema={
          "features": [
            {"name": c, "type": "float"} for c in X.columns
          ],
          "prediction": "numeric",
        },
        custom_metrics=[{"name": "mean_square_error", "type": "metric", "value_type": "float"}]
    )
    print(f"Logged model: {model_version.fqn}")
    run.end()


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--kernel", default="linear", type=str)
    args = parser.parse_args()

    train(kernel=args.kernel)

In [ ]:
%%writefile requirements.txt
pandas==1.3.5
scikit-learn==1.2.1
mlfoundry==0.6.1
matplotlib==3.2.2

In [ ]:
import logging
from getpass import getpass

from servicefoundry import Build, Job, PythonBuild

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s', force=True)

job = Job(
    name="train",
    image=Build(
        build_spec=PythonBuild(
            command="python train.py",
            requirements_path="requirements.txt",
        )
    ),
    env={
        "TFY_API_KEY": TFY_API_KEY
    }
)
deployment = job.deploy(workspace_fqn=WORKSPACE)